## Primeiro Experimento: Comparação do m² com a Cidade Inteira

In [ ]:
target = ps.NumericTarget('valor_m2')

include = ['Bairro', 'Tipo Construtivo Preponderante', 'Ano Avaliacao', 'idade']
ignore = list(set(list(itbi)) - set(include))
search_space = ps.create_selectors(itbi, ignore=ignore)

# Usaremos a StandardQF, que mede a diferença da média do subgrupo em desvios padrão
# O parâmetro 'a' ajuda a ponderar o tamanho do subgrupo.
quality_function = ps.StandardQFNumeric(a=0.5)

In [ ]:
# Criando a tarefa de descoberta de subgrupos
task = ps.SubgroupDiscoveryTask(
    itbi,
    target,
    search_space,
    result_set_size=10,  # 10 melhores subgrupos
    depth=3,             # Profundidade máxima da descrição 
    qf=quality_function
)

# Executando a busca
result = ps.BeamSearch().execute(task)

In [ ]:
media_global_nominal = itbi['valor_m2'].mean()
media_global_real = itbi['valor_m2_corrigido'].mean()
df_results = result.to_dataframe()
medias_reais_sg = []

for index, row in df_results.iterrows():
    subgrupo_obj = row['subgroup']
    dados_do_subgrupo = subgrupo_obj.covers(itbi)
    media_real = itbi[dados_do_subgrupo]['valor_m2_corrigido'].mean()
    medias_reais_sg.append(media_real)
df_results['mean_sg_corrigido'] = medias_reais_sg

print("="*60)
print("Análise Detalhada dos Subgrupos de Maior Impacto Nominal")
print("="*60)
print(f"Média Global Nominal: R$ {media_global_nominal:,.2f}")
print(f"Média Global Real (corrigida): R$ {media_global_real:,.2f}\n")


for index, row in df_results.iterrows():
    # Coletando os valores da linha
    qualidade = row['quality']
    descricao = row['subgroup']
    tamanho_subgrupo = row['size_sg']
    media_nominal_sg = row['mean_sg']
    media_real_sg = row['mean_sg_corrigido']

    # Calculando os dois impactos percentuais
    impacto_nominal = ((media_nominal_sg / media_global_nominal) - 1) * 100
    impacto_real = ((media_real_sg / media_global_real) - 1) * 100

    # Imprimindo o "card" de resultado para cada subgrupo
    print(f"Qualidade: {qualidade:.3f} | Subgrupo: {descricao}")
    print(f"   > Tamanho: {tamanho_subgrupo:,.0f} imóveis")
    print(f"   > Média Nominal: R$ {media_nominal_sg:,.2f}  (Impacto: {impacto_nominal:+.2f}%)")
    print(f"   > Média Real   : R$ {media_real_sg:,.2f}  (Impacto: {impacto_real:+.2f}%)")
    print("-" * 40)